# PostgreSQL EV Database Initialization App 

#### This App is going to create and add data to a database in PostgreSQL called "ev_db" to hold data for electric vehicles registered in Washington State and the income data for all associated counties in the year selected. 

In [1]:
# Import Dependencies
import psycopg2
import pandas as pd
import requests
import json
from datetime import datetime

# Import postgres server info key and BEA API Key
from postgres_info import user, password
from api_keys import bea_key

#### Extract the Income by County data from the Bureau of Economic Analysis API and assemble it into a pandas dataframe to be uploaded to SQL later.

In [2]:
# Create a 'while True:' statement to collect the year variable in the appropriate format
while True:
    # Create a variable to hold the current year as an integer
    current_year = int(datetime.now().year)
    # Create a variable to store the user input for the year to be extracted from the API
    year_parameter = input(f"Enter a year to pull for county income from the BEA API (must be a single year after 2009 and before {current_year}): ")
    # Use an 'if' statement to ensure the input is a 4 character digit 
    if year_parameter.isdigit() and len(year_parameter) == 4:
        # Convert user input variable to integer if meets format requirements
        year_parameter_int = int(year_parameter)
        # Use another 'if' statement to ensure the input year is between 2010 and the curent year
        if 2010 <= year_parameter_int < current_year:
            # Break the 'while True:' statement if conditions are met
            break
        # If input is not between 2010 and current year print error
        else:
            print(f"Invalid input. Please choose a year after 2009, but before {current_year}")
    # 'If' Statement to handle when user types quit
    if year_parameter == 'quit':
        # Exit entire python script
        exit()
    # 'Else' statement to handle error if user types a non-digit and not 4 digits
    else:
        print("Invalid input. Please enter exactly four digits.")

# Create variables for the API call to get Income data by county
base_url = 'https://apps.bea.gov/api/data'
tablename = 'CAINC1' #per capita income by county
linecode = '1' 

# Print statement confirming that API is being extracted with selected year
print(f'Pulling Income data from API for year {year_parameter}...')

# Create URL to Bureau of Economic Analysis for per capita income by county by year
income_url = f'{base_url}?&UserID={bea_key}&method=GetData&datasetname=Regional&TableName={tablename}&LineCode={linecode}&Year={year_parameter}&GeoFips=COUNTY&ResultFormat=json'

# Pull the response into the notebook
income_json = requests.get(income_url).json()

# Print statement to confirm that income data was eextracted successfully
print(f'Income data for year {year_parameter} extracted successfully!')

Pulling Income data from API for year 2023...
Income data for year 2023 extracted successfully!


In [3]:
# Display the result as a dataframe with 'json_normalize()' using only relevant columns 
json_df = pd.json_normalize(income_json['BEAAPI']['Results']['Data'])

# Create a new dataframe with columns: 'GeoFips','GeoName','TimePeriod', and 'DataValue'
income_df = json_df[['GeoFips','GeoName','TimePeriod','DataValue']]

# Rename columns
income_df = income_df.rename(columns={'GeoFips':'FIPS code','TimePeriod':'Income Year','GeoName':'County & State','DataValue':'PerCapita Income'})

# Format 'Year' column to be a string
income_df['Income Year'] = income_df['Income Year'].astype(str)

# Format 'FIPS code' column to be a string 
income_df['FIPS code'] = income_df['FIPS code'].astype(str)
# Use 'lambda' function to add leading zero back to the code
income_df['FIPS code'] = income_df['FIPS code'].apply(lambda x: x.zfill(5))

# Use regex to extract the county from the 'County & State' field
income_df['County'] = income_df['County & State'].str.extract(r'^([^,]+)')
# Use regex to extract the state from the 'County & State' field
income_df['State'] = income_df['County & State'].str.extract(r',([^,]+)$')
# Remove '*' from the end of the state value if it exists
income_df['State'] = income_df['State'].str.rstrip('*')
# Remove ' ' from the beginning of the state value if it exists
income_df['State'] = income_df['State'].str.lstrip()
# Drop old 'County & State' column
income_df = income_df.drop(columns=['County & State'])

# Convert PerCapita Income to Integer
income_df['PerCapita Income'] = income_df['PerCapita Income'].astype(int)
# Divide by hundred to get true value then convert to float
income_df['PerCapita Income'] = (income_df['PerCapita Income']/100).astype(float)

# Display the dataframe
income_df

,FIPS code,Income Year,PerCapita Income,County,State
0,01001,2023,32029.10,Autauga,AL
1,01003,2023,154560.83,Baldwin,AL
2,01005,2023,10210.34,Barbour,AL
3,01007,2023,8711.12,Bibb,AL
4,01009,2023,26929.60,Blount,AL
...,...,...,...,...,...
3135,56037,2023,26446.67,Sweetwater,WY
3136,56039,2023,109597.23,Teton,WY
3137,56041,2023,10237.74,Uinta,WY
3138,56043,2023,4431.28,Washakie,WY


#### Use csv in Resources directory to beild a dataframe with the Electric Vehicle Population Data for vehicles registered in Washington state. Will be used as the base dataframe to be divided up

In [4]:
# Upload 'Electric Vehicle Population Data' csv to dataframe
main_df = pd.read_csv("Resources/Electric_Vehicle_Population_Data.csv")

# Drop rows where the postal code is 'NaN'
main_df = main_df.dropna(subset=['Postal Code'])

# Convert 'DOL Vehicle ID' and 'Model Year' columns to a string
main_df[['DOL Vehicle ID','Model Year']] = main_df[['DOL Vehicle ID','Model Year']].astype(str)

# Convert '2020 Census Tract' and 'Postal Code' columns to a string from a float
main_df[['2020 Census Tract','Postal Code']] = main_df[['2020 Census Tract','Postal Code']].astype(int).astype(str)

# Convert 'Legislative District' column to string and then remove decimal if exists with regex
main_df['Legislative District'] = main_df['Legislative District'].astype(str).str.replace(r'\.0$', '', regex=True)
# Replace 'nan' in Legislative District with empty value
main_df['Legislative District'] = main_df['Legislative District'].str.replace('nan','')

# Replace the first 6 charcters in the VIN column with 'xxxxxx' to help protect privacy
main_df['VIN (1-10)'] = main_df['VIN (1-10)'].str.replace(r'^.{6}', 'xxxxxx', regex=True)

# Extract longitude from 'Vehicle Location' column and convert to float
main_df['Longitude'] = main_df['Vehicle Location'].str.extract(r'(-\d+\.\d+)', expand=False).astype(float)
# Extract latitude from 'Vehicle Location' column and convert to float
main_df['Latitude'] = main_df['Vehicle Location'].str.extract(r' (\d+\.\d+)', expand=False).astype(float)

# Drop 'Vehicle Location' column
main_df = main_df.drop('Vehicle Location', axis=1)

# Merge income data frome above dataframe 'income_df'
new_main_df = pd.merge(main_df, income_df, on=['County','State'])

# Display dataframe
new_main_df

,VIN (1-10),County,City,State,Postal Code,Model Year,Make,Model,Electric Vehicle Type,Clean Alternative Fuel Vehicle (CAFV) Eligibility,...,Base MSRP,Legislative District,DOL Vehicle ID,Electric Utility,2020 Census Tract,Longitude,Latitude,FIPS code,Income Year,PerCapita Income
0,xxxxxxCP8D,King,Shoreline,WA,98177,2013,NISSAN,LEAF,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,...,0.0,32,125450447,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033020100,-122.36498,47.72238,53033,2023,2776422.67
1,xxxxxxE45K,King,Seattle,WA,98112,2019,TESLA,MODEL S,Battery Electric Vehicle (BEV),Clean Alternative Fuel Vehicle Eligible,...,0.0,43,101662900,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA),53033006300,-122.30207,47.64085,53033,2023,2776422.67
2,xxxxxxE28M,Kitsap,Olalla,WA,98359,2021,VOLKSWAGEN,ID.4,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,...,0.0,26,272118717,PUGET SOUND ENERGY INC,53035092803,-122.54729,47.42602,53035,2023,205026.25
3,xxxxxxFP6H,Thurston,Olympia,WA,98501,2017,TOYOTA,PRIUS PRIME,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,0.0,22,349372929,PUGET SOUND ENERGY INC,53067010400,-122.89166,47.03956,53067,2023,197017.95
4,xxxxxxCU9G,Thurston,Olympia,WA,98506,2016,FORD,C-MAX,Plug-in Hybrid Electric Vehicle (PHEV),Not eligible due to low battery range,...,0.0,22,171625653,PUGET SOUND ENERGY INC,53067010300,-122.87741,47.05997,53067,2023,197017.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216735,xxxxxxE44D,Douglas,East Wenatchee,WA,98802,2013,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,0.0,12,122822822,PUD NO 1 OF DOUGLAS COUNTY,53017950400,-120.29473,47.41515,53017,2023,23261.72
216736,xxxxxxLF9R,Whatcom,Bellingham,WA,98229,2024,KIA,NIRO,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,0.0,40,267143887,PUGET SOUND ENERGY INC||PUD NO 1 OF WHATCOM CO...,53073001202,-122.45486,48.74490,53073,2023,144009.63
216737,xxxxxxEE9R,King,Redmond,WA,98052,2024,TESLA,MODEL Y,Battery Electric Vehicle (BEV),Eligibility unknown as battery range has not b...,...,0.0,48,274988388,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA),53033032325,-122.13158,47.67858,53033,2023,2776422.67
216738,xxxxxxE49D,Pierce,Gig Harbor,WA,98329,2013,CHEVROLET,VOLT,Plug-in Hybrid Electric Vehicle (PHEV),Clean Alternative Fuel Vehicle Eligible,...,0.0,26,117353064,BONNEVILLE POWER ADMINISTRATION||CITY OF TACOM...,53053072601,-122.66580,47.38336,53053,2023,605402.83


#### Create a new dataframe from the above 'new_main_df' using location based columns. This will create a new dataframe that stores unique location data by zip code

In [5]:
# Create dataframe with location based columns from above dataframe
vehicle_location_df = new_main_df[['Postal Code','FIPS code','County','City','State','Legislative District','Latitude','Longitude','2020 Census Tract']]

# Get unique values by dropping postal code duplicates
location_df = vehicle_location_df.drop_duplicates(subset='Postal Code').sort_values(by=['Postal Code'], ascending=False)

# Save as a new csv
location_df.to_csv("Output_CSVs/location.csv", index=False)

# Display the Dataframe
location_df

,Postal Code,FIPS code,County,City,State,Legislative District,Latitude,Longitude,2020 Census Tract
61398,99403,53003,Asotin,Clarkston,WA,9,46.41402,-117.04556,53003960300
39524,99402,53003,Asotin,Asotin,WA,9,46.34056,-117.04784,53003960100
214797,99371,53001,Adams,Washtucna,WA,9,46.75578,-118.31100,53001950200
99,99362,53071,Walla Walla,Walla Walla,WA,16,46.07068,-118.34261,53071920901
98727,99361,53071,Walla Walla,Waitsburg,WA,16,46.27013,-118.15448,53071920100
...,...,...,...,...,...,...,...,...,...
175430,12866,36091,Saratoga,Saratoga Springs,NY,,43.08412,-73.78441,36091061102
205108,12833,36091,Saratoga,Greenfield Center,NY,,43.12840,-73.84643,36091060601
178073,11572,36059,Nassau,Oceanside,NY,,40.64212,-73.64036,36059413200
153457,11209,36047,Kings,Brooklyn,NY,,40.62956,-74.03151,36047013800


#### Create a new datarame to store unique vehicle models

In [6]:
# Create dataframe with vehicle info columns. Sort by year and drop duplicates
vehicle_types_df = main_df[['Model Year','Make','Model','Electric Vehicle Type']].sort_values(by=['Model Year','Make','Model'], ascending=True).drop_duplicates()

# Save as csv and reupload to reset index
vehicle_types_df.to_csv("Output_CSVs/vehicle_types.csv", index=False)
vehicle_types_df = pd.read_csv("Output_CSVs/vehicle_types.csv")

# Re-convert Model Year column to a string
vehicle_types_df['Model Year'] = vehicle_types_df['Model Year'].astype(str)

# Create a new column to hold new vehicle id. Use the columns index and add one digit
vehicle_types_df['Vehicle Type ID'] = vehicle_types_df.index+1
# Use 'lambda' function to add leading zeros to ID to ensure it is four digits
vehicle_types_df['Vehicle Type ID'] = vehicle_types_df['Vehicle Type ID'].astype(str).apply(lambda x: x.zfill(4))
# Add 'vm' to the front of every ID
vehicle_types_df['Vehicle Type ID'] = vehicle_types_df['Vehicle Type ID'].apply(lambda x: f'vm{x}')

# Move new ID column to the front of dataframe
vehicle_types_df = vehicle_types_df[['Vehicle Type ID'] + [col for col in vehicle_types_df.columns if col != 'Vehicle Type ID']]

# Resave as CSV
vehicle_types_df.to_csv("Output_CSVs/vehicle_types.csv", index=False)

# Display the dataframe
vehicle_types_df

,Vehicle Type ID,Model Year,Make,Model,Electric Vehicle Type
0,vm0001,1999,FORD,RANGER,Battery Electric Vehicle (BEV)
1,vm0002,2000,FORD,RANGER,Battery Electric Vehicle (BEV)
2,vm0003,2002,TOYOTA,RAV4,Battery Electric Vehicle (BEV)
3,vm0004,2003,TOYOTA,RAV4,Battery Electric Vehicle (BEV)
4,vm0005,2008,TESLA,ROADSTER,Battery Electric Vehicle (BEV)
...,...,...,...,...,...
553,vm0554,2025,VOLVO,EX40,Battery Electric Vehicle (BEV)
554,vm0555,2025,VOLVO,S60,Plug-in Hybrid Electric Vehicle (PHEV)
555,vm0556,2025,VOLVO,V60,Plug-in Hybrid Electric Vehicle (PHEV)
556,vm0557,2025,VOLVO,XC60,Plug-in Hybrid Electric Vehicle (PHEV)


#### Craete a new dataframe to store unique values for the CAFV Eligibility column

In [7]:
# Create CAFV Eligibility table using 'CAFV eligibility' column and drop duplicates
cafv_df = main_df[['Clean Alternative Fuel Vehicle (CAFV) Eligibility']].drop_duplicates()
# Save as CSV and reupload to reset index
cafv_df.to_csv("Output_CSVs/csfv_eligibility.csv", index=False)
cafv_df = pd.read_csv("Output_CSVs/csfv_eligibility.csv")

# Create new column with index for ID and add 1
cafv_df['CAFV Eligibility ID'] = cafv_df.index+1
# Add 'cafv' to the front of the id numbers using 'lambda' function
cafv_df['CAFV Eligibility ID'] = cafv_df['CAFV Eligibility ID'].apply(lambda x: f'cafv{x}')

# Move ID First
cafv_df = cafv_df[['CAFV Eligibility ID','Clean Alternative Fuel Vehicle (CAFV) Eligibility']]

# Resave as CSV
cafv_df.to_csv("Output_CSVs/csfv_eligibility.csv", index=False)

# Display the dataframe
cafv_df

,CAFV Eligibility ID,Clean Alternative Fuel Vehicle (CAFV) Eligibility
0,cafv1,Clean Alternative Fuel Vehicle Eligible
1,cafv2,Eligibility unknown as battery range has not b...
2,cafv3,Not eligible due to low battery range


#### Create a new dataframe to get unique electric utility companies

In [8]:
# Create a new dataframe with the utility company name and drop duplicates
utilities_df = main_df[['Electric Utility']].drop_duplicates()

# Save as CSV and reopen to reset index
utilities_df.to_csv("Output_CSVs/utilities.csv", index=False)
utilities_df = pd.read_csv("Output_CSVs/utilities.csv")

# Create new column with index for ID and add 1
utilities_df['Electric Company ID'] = utilities_df.index+1

# Use 'lambda' function to add leading zeros to ID
utilities_df['Electric Company ID'] = utilities_df['Electric Company ID'].astype(str).apply(lambda x: x.zfill(3))
# Add 'cafv' to the front of the id numbers
utilities_df['Electric Company ID'] = utilities_df['Electric Company ID'].apply(lambda x: f'uc{x}')

# Move ID column First
utilities_df = utilities_df[['Electric Company ID','Electric Utility']]

# Resave as CSV
utilities_df.to_csv("Output_CSVs/utilities.csv", index=False)

# Display the dataframe
utilities_df

,Electric Company ID,Electric Utility
0,uc001,CITY OF SEATTLE - (WA)|CITY OF TACOMA - (WA)
1,uc002,PUGET SOUND ENERGY INC
2,uc003,PACIFICORP
3,uc004,PUGET SOUND ENERGY INC||CITY OF TACOMA - (WA)
4,uc005,PUD NO 2 OF GRANT COUNTY
...,...,...
70,uc071,BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF F...
71,uc072,BONNEVILLE POWER ADMINISTRATION||BENTON RURAL ...
72,uc073,BONNEVILLE POWER ADMINISTRATION||PUD NO 1 OF A...
73,uc074,BONNEVILLE POWER ADMINISTRATION||NESPELEM VALL...


#### Remake the income dataframe to drop the county and state values. The FIPS code will be the unique identifier

In [9]:
# Remake income_df without county and state fields
income_df = income_df[['FIPS code','Income Year','PerCapita Income']]

# Resave to existing csv 
income_df.to_csv('Output_CSVs/income_data.csv',index=False)

# Display the new dataframe
income_df

,FIPS code,Income Year,PerCapita Income
0,01001,2023,32029.10
1,01003,2023,154560.83
2,01005,2023,10210.34
3,01007,2023,8711.12
4,01009,2023,26929.60
...,...,...,...
3135,56037,2023,26446.67
3136,56039,2023,109597.23
3137,56041,2023,10237.74
3138,56043,2023,4431.28


#### Merge above tables together with the 'new_main_df' from earlier to create one large dataframe. Then, recreate a new dataframe to store unique registered vehicles using only the ID columns from the above dataframes and the unique vehicle fields, such as 'Electric Range', 'Base MSRP, 'VIN', and 'DOL Vehicle ID'

In [10]:
# Merge vehicle types dataframe with the 'new_main_df' 
vehicle_df = pd.merge(new_main_df,vehicle_types_df, on=['Model Year','Make','Model','Electric Vehicle Type'])

# Merge the CAFV eligibility dataframe with the new merged dataframe above
vehicle_df = pd.merge(vehicle_df,cafv_df, on='Clean Alternative Fuel Vehicle (CAFV) Eligibility')

# Merge the Utilities dataframe with the new merged dataframe above to create one large dataframe
vehicle_df = pd.merge(vehicle_df,utilities_df, on='Electric Utility')

# Remake DataFrame with needed columns only
vehicle_df = vehicle_df[['DOL Vehicle ID','Vehicle Type ID','Postal Code','FIPS code','Electric Company ID','CAFV Eligibility ID','VIN (1-10)','Electric Range','Base MSRP']]

# Save as csv
vehicle_df.to_csv("Output_CSVs/vehicle.csv", index=False)

# Display the Dataframe
vehicle_df

,DOL Vehicle ID,Vehicle Type ID,Postal Code,FIPS code,Electric Company ID,CAFV Eligibility ID,VIN (1-10),Electric Range,Base MSRP
0,125450447,vm0027,98177,53033,uc001,cafv1,xxxxxxCP8D,75.0,0.0
1,101662900,vm0199,98112,53033,uc001,cafv1,xxxxxxE45K,270.0,0.0
2,272118717,vm0288,98359,53035,uc002,cafv2,xxxxxxE28M,0.0,0.0
3,349372929,vm0129,98501,53067,uc002,cafv3,xxxxxxFP6H,25.0,0.0
4,171625653,vm0081,98506,53067,uc002,cafv3,xxxxxxCU9G,19.0,0.0
...,...,...,...,...,...,...,...,...,...
216735,122822822,vm0021,98802,53017,uc025,cafv1,xxxxxxE44D,38.0,0.0
216736,267143887,vm0479,98229,53073,uc023,cafv1,xxxxxxLF9R,33.0,0.0
216737,274988388,vm0518,98052,53033,uc004,cafv2,xxxxxxEE9R,0.0,0.0
216738,117353064,vm0021,98329,53053,uc022,cafv1,xxxxxxE49D,38.0,0.0


#### Convert dataframes to lists to be stored in PostgreSQL tables

In [11]:
# Convert above dataframes to a list to be inserted into SQL table
cafv_list = cafv_df.values.tolist()
utilities_list = utilities_df.values.tolist()
income_list = income_df.values.tolist()
location_list = location_df.values.tolist()
vehicle_types_list = vehicle_types_df.values.tolist()
vehicles_list = vehicle_df.values.tolist()


#### Establish a connection to PostgreSQL to create a new database called 'ev_db'

In [12]:
# Use a 'try' statement to handle errors connecting to the database
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="postgres")
    # Make sure autocommit is 'True' to commit new database successfully
    conn.autocommit = True
# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)
# Use 'else' statemnt is connection if successful
else:
    print('-'*60)
    print("Connection established successfully")

# Create cursor
cur = conn.cursor()

# Query to create 'ev_db' database
sql = '''CREATE DATABASE ev_db;'''

# Use 'try' statement to execute above SQL statement
print('-'*60)
try:
    cur.execute(sql)
    print("Database successfully created!")
# Use 'except' statement to handle if database already exists
except:
    print("Database already exists! Execute command aborted...")


# Close cursor
cur.close()

# Close connection
conn.close()

------------------------------------------------------------
Connection established successfully
------------------------------------------------------------
Database already exists! Execute command aborted...


#### Reconnect to PostgreSQL to add tables and data to new 'ev_db' database

In [13]:
# Use a 'try' statement to handle errors connecting to the database
try:
    # Connect to the database
    conn = psycopg2.connect(
        host="127.0.0.1",
        port="5432",
        user=user,
        password=password,
        database="ev_db")
# Use 'except' statement to handle and display an error connect to PostgreSQL
except psycopg2.Error as e:
    print('-'*60)
    print("Error connecting to the database:")
    print(e)
# Use 'else' statemnt if connection is successful
else:
    print('-'*60)
    print("Connection re-established successfully")

# Create cursor
cur = conn.cursor() 

# SQL queries to drop tables if already exists. Use 'CASCADE' to ensure all dependent tables are dropped
drop_cafv_table_query = '''DROP TABLE IF EXISTS cafv_eligibility CASCADE;'''
drop_utilities_query = '''DROP TABLE IF EXISTS utility_companies CASCADE'''
drop_income_table_query = '''DROP TABLE IF EXISTS county_income CASCADE'''
drop_location_info_query = '''DROP TABLE IF EXISTS location_info CASCADE;'''
drop_vehicle_types_query = '''DROP TABLE IF EXISTS vehicle_types CASCADE;'''
drop_vehicles_query = '''DROP TABLE IF EXISTS vehicles CASCADE;'''

# SQL queries to create tables for our data
create_cafv_table_query = '''
    CREATE TABLE cafv_eligibility (
        cafv_id VARCHAR(5) PRIMARY KEY
        CHECK (cafv_id LIKE 'cafv%'),
        cafv_eligibility VARCHAR
    );
    '''

create_utilities_table_query = '''
    CREATE TABLE utility_companies (
        utility_company_id VARCHAR(5) PRIMARY KEY
        CHECK (utility_company_id LIKE 'uc%'),
        utility_company_name VARCHAR
    );
    '''

create_income_table_query = '''
    CREATE TABLE county_income (
        fips_code VARCHAR(5) PRIMARY KEY,
        income_year VARCHAR(4) NOT NULL,
        percapita_income DECIMAL(10,2)
    );    
    '''

create_location_info_query = """
    CREATE TABLE location_info (
        postal_code VARCHAR(5) PRIMARY KEY,
        fips_code VARCHAR(5),
        FOREIGN KEY (fips_code) REFERENCES county_income (fips_code),
        county VARCHAR NOT NULL,
        city VARCHAR NOT NULL,
        state VARCHAR(2) NOT NULL,
        legislative_district VARCHAR,
        latitude DECIMAL(10,6) NOT NULL,
        longitude DECIMAL(10,6) NOT NULL,
        census_tract_2020 VARCHAR
    );
    """

create_vehicle_types_query = """
    CREATE TABLE vehicle_types (
        vehicle_type_id VARCHAR(6) PRIMARY KEY
        CHECK (vehicle_type_id LIKE 'vm%'),
        model_year VARCHAR(4) NOT NULL,
        make VARCHAR NOT NULL,
        model VARCHAR NOT NULL,
        ev_type VARCHAR
    );
    """

create_vehicles_query = """
    CREATE TABLE vehicles (
        dol_vehicle_id VARCHAR(9) PRIMARY KEY,
        vehicle_type_id VARCHAR(6) NOT NULL
        CHECK (vehicle_type_id LIKE 'vm%'),
        FOREIGN KEY (vehicle_type_id) REFERENCES vehicle_types (vehicle_type_id),
        postal_code VARCHAR(5) NOT NULL,
        FOREIGN KEY (postal_code) REFERENCES location_info (postal_code),
        fips_code VARCHAR(5),
        FOREIGN KEY (fips_code) REFERENCES county_income (fips_code),
        utility_company_id VARCHAR(5),
        CHECK (utility_company_id LIKE 'uc%'),
        FOREIGN KEY (utility_company_id) REFERENCES utility_companies (utility_company_id),
        cafv_id VARCHAR(5)
        CHECK (cafv_id LIKE 'cafv%'),
        FOREIGN KEY (cafv_id) REFERENCES cafv_eligibility (cafv_id),
        vin VARCHAR(10) NOT NULL
        CHECK (vin LIKE 'xxxxxx%'),
        electric_range DECIMAL(10,1),
        base_msrp DECIMAL(10,2)
    );
    """


# Execute Drop table query 
cur.execute(drop_cafv_table_query)
# Execute the CREATE TABLE query
cur.execute(create_cafv_table_query)
print('-'*60)
print("CAFV table created or recreated successfully!")

# Execute Drop table query 
cur.execute(drop_utilities_query)
# Execute the CREATE TABLE query
cur.execute(create_utilities_table_query)
print("Utility Company table created or recreated successfully!")

# Execute Drop table query 
cur.execute(drop_income_table_query)
# Execute the CREATE TABLE query
cur.execute(create_income_table_query)
print("County Income table created or recreated successfully!")

# Execute Drop table query 
cur.execute(drop_location_info_query)
# Execute the CREATE TABLE query
cur.execute(create_location_info_query)
print("Location info table created or recreated successfully!")

# Execute Drop table query 
cur.execute(drop_vehicle_types_query)
# Execute the CREATE TABLE query
cur.execute(create_vehicle_types_query)
print("Vehicle types table created or recreated successfully!")

# Execute Drop table query 
cur.execute(drop_vehicles_query)
# Execute the CREATE TABLE query
cur.execute(create_vehicles_query)
print("Vehicles table created or recreated successfully!")

------------------------------------------------------------
Connection re-established successfully
------------------------------------------------------------
CAFV table created or recreated successfully!
Utility Company table created or recreated successfully!
County Income table created or recreated successfully!
Location info table created or recreated successfully!
Vehicle types table created or recreated successfully!
Vehicles table created or recreated successfully!


#### Insert all values from six lists from above to their respective tables in the SQL database

In [14]:
# Print statement showing data is loading
print('-'*60)
print('Loading CAFV Eligibility data into table...')
# Use a 'for' loop to insert each row of data into 'cafv_eligibility' table
for row in cafv_list:
    sql = "INSERT INTO cafv_eligibility (cafv_id,cafv_eligibility) VALUES (%s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('CAFV Elgibility data added successfully!')

# Print statement showing data is loading
print('-'*60)
print('Loading Utility Company data into table...')
# Use a for loop to insert each row of data into location table
for row in utilities_list:
    sql = "INSERT INTO utility_companies (utility_company_id,utility_company_name) VALUES (%s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('Utility Company data added successfully!')

# Print statement showing data is loading
print('-'*60)
print('Loading County Income data into table...')
# Use a for loop to insert each row of data into location table
for row in income_list:
    sql = "INSERT INTO county_income (fips_code,income_year,percapita_income) VALUES (%s, %s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('County Income data added successfully!')

# Print statement showing data is loading
print('-'*60)
print('Loading Location Info data into table...')
# Use a for loop to insert each row of data into location table
for row in location_list:
    sql = "INSERT INTO location_info (postal_code,fips_code,county,city,state,legislative_district,latitude,longitude,census_tract_2020) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('Location Info data added successfully!')

# Print statement showing data is loading
print('-'*60)
print('Loading Vehicle Types data into table...')
# Use a for loop to insert each row of data into vehicle info table
for row in vehicle_types_list:
    sql = "INSERT INTO vehicle_types (vehicle_type_id,model_year,make,model,ev_type) VALUES (%s, %s, %s, %s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('Vehicle Types data added successfully!')

# Print statement showing data is loading
print('-'*60)
print('Loading Vehicle data into table...')
# Use a for loop to insert each row of data into main vehicle table
for row in vehicles_list:
    sql = "INSERT INTO vehicles (dol_vehicle_id,vehicle_type_id,postal_code,fips_code,utility_company_id,cafv_id,vin,electric_range,base_msrp) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cur.execute(sql, row)
# Print statment showing data loaded successfully
print('Vehicle data added successfully!')

# Commit all the changes to the database
conn.commit()

# Close the cursor
cur.close()

# Close the connection
conn.close()

# Print when database initialization is done
print('-'*60)
print('Database initalization is complete. Please check ev_db database in PostGRES to ensure data loaded correctly. Thank you for using our app.')

------------------------------------------------------------
Loading CAFV Eligibility data into table...
CAFV Elgibility data added successfully!
------------------------------------------------------------
Loading Utility Company data into table...
Utility Company data added successfully!
------------------------------------------------------------
Loading County Income data into table...
County Income data added successfully!
------------------------------------------------------------
Loading Location Info data into table...
Location Info data added successfully!
------------------------------------------------------------
Loading Vehicle Types data into table...
Vehicle Types data added successfully!
------------------------------------------------------------
Loading Vehicle data into table...
Vehicle data added successfully!
------------------------------------------------------------
Database initalization is complete. Please check ev_db database in PostGRES to ensure data loade